# Image registration plugin

Create a plugin to register two images enabling users to choose from various algorithms 
1) Read in two images
2) Select registration algorithm
3) Warp moving image by modifying metadata of image layer
4) Display result in napari viewer


In [30]:
from skimage import data
import numpy as np
from scipy import ndimage

import napari
from magicgui import magicgui

In [31]:
IMAGE2D = data.cells3d()[30,1]

trafo_matrix = np.eye(3)
trafo_matrix[0,2] = -22.4
trafo_matrix[1,2] = 13.2    

inv_mat = np.linalg.inv(trafo_matrix)
IMAGE2D_offset = ndimage.affine_transform(input=IMAGE2D,
                                          matrix=inv_mat,
                                          output_shape=IMAGE2D.shape,
                                          order=2)

In [32]:
@magicgui
def register_images(
    fixed_image: 'napari.layers.Image', 
    moving_image: 'napari.layers.Image') -> 'napari.layers.Image':
    
    from skimage.registration import phase_cross_correlation 
    from napari.layers import Image

    shift, __, __ = phase_cross_correlation(fixed_image.data, moving_image.data)

    trafo_matrix = np.eye(3)
    trafo_matrix[0,2] = shift[0]
    trafo_matrix[1,2] = shift[1]
    
    warped_image_data = moving_image.data.copy()
    warped_image = Image(warped_image_data)
    warped_image.affine.affine_matrix = trafo_matrix
    warped_image.name = moving_image.name + '_warped'
    
    return warped_image

viewer = napari.view_image(IMAGE2D)
viewer.add_image(IMAGE2D_wrp)

viewer.window.add_dock_widget(register_images)